_To use this notebook you must have a (free) FEO account, available [here](https://feo.transitionzero.org/). You may then install the feo-client library, if necessary, and then use the package to log in. You can also run notebook in a Colab notebook via the badge below._

<a target="_blank" href="https://colab.research.google.com/github/transition-zero/feo-client-examples/blob/main/feo-client-examples/0_nodes.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ feo-client

In [ ]:
from feo.client.auth import login

login()

## Geospatial

_Object-oriented interface for Nodes via the Python client_

FEO comprises a wide variety of geospatial data, covering vector and raster data types. Vector types include `node geometries`, `hydrobasins`, `exclusive economic zones` and `protected areas`. Each of these exists as a `Collection` that can be browsed and searched via several methods.

In this notebook we will gather some vector data for a node, namely:
- The node geometry
- The protected areas intersecting the node geometry
- Any corresponding offshore areas (Exclusive Economic Zones)

In [ ]:
from feo.client import Node, Features

In [ ]:
# Let's use Bali
node_name = "Bali"
nodes = Node.search(node_name)
try:
    bali = nodes[0]
    print(bali)
except IndexError:
    raise IndexError(f"No nodes found for {node_name}")

In [ ]:
bali_geom = bali.geometry

In [ ]:
type(bali_geom)

The `geometry` property is a `Geometry` class. This class provides convenience methods for downstream purposes. In the base `feo-client` install you can convert the geometry to a dictonary or a GeoJSON string

In [ ]:
print(bali_geom.to_geojson()[:100]) # sliced to limit output

If you installed the `geo` requirements you'll be able to also use the `to_shape` method to return a `shapely.Geometry` object.

In [ ]:
bali_geom.to_shape()

### Search for other vector data
Next we'll search for any protected areas located within Bali.

In [ ]:
# Search for Protected Areas
pa_feats = Features.search(collection_id="biodiversity-wdpa", geometry=bali_geom)

`pa_feats` is a `Features` objects; a collection of features, each with properties and a geometry

In [ ]:
# How many protected areas were returned?
len(pa_feats.features)

In [ ]:
# Examine a feature

In [ ]:
ft = pa_feats.features[0]
ft.properties

In [ ]:
ft.geometry.to_shape()

If you have the `geo` dependencies installed you can convert the `Features` object to a `GeoDataFrame`

In [ ]:
gdf = pa_feats.to_geodataframe()
gdf

### Visualise the data
`Features` and `Geometry` objects both have GeoJSON representations, allowing for easy interoperability with other geospatial tools. In this example, we use `folium` to plot an interactive map of Bali and its protected areas.

In [ ]:
!pip install folium

In [ ]:
import folium
from folium.features import GeoJson

In [ ]:
# Define the layer styles
bali_style = {"fillColor": "#f2ff00", "color": "#f2ff00"}
pa_style = {'fillColor': '#000000', 'color': '#000000'}

In [ ]:
bali_geom_shape = bali_geom.to_shape() # convert our Geometry object to a shapely object
map_center = bali_geom_shape.centroid # access the centroid using the shape property

# Configure the map
map = folium.Map(location=(map_center.y, map_center.x), zoom_start=10)

# Add vector data to the map via the GeoJSON interface
map.add_child(GeoJson(bali_geom.to_geojson(), name="Bali", style_function=lambda x: bali_style))
map.add_child(GeoJson(pa_feats.to_geojson(), name="World Database on Protected Areas",
                      style_function=lambda x: pa_style,
                      popup=folium.GeoJsonPopup(fields=["name", "designation"]))
)
folium.LayerControl().add_to(map)
map